# Setup Notebook

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import bokeh.plotting
import types

assert sys.version_info > (3,8)

root_folder = os.path.join(os.getcwd().split('text_analytics')[0], 'text_analytics')
corpus_folder = os.path.join(root_folder, 'data')

sys.path = sys.path + [ root_folder, globals()['_dh'][-1] ]

import text_analytic_tools.utility.utils as utility
from corpus_tools import get_textacy_corpus
from word_trends_textacy_corpus_gui import vectorize_textacy_corpus

bokeh.plotting.output_notebook(hide_banner=True)

container = types.SimpleNamespace(corpus=None, t_corpus=None, index=None, handle=None, data_source=None, data=None, figure=None)


### Load and display corpus
<!--
The corpus was created with the following settings:
 - Tokens were converted to lower case.
 - Only tokens that contains at least one alphanumeric character (isalnum).
 - Accents are ot removed (deacc)
 - Min token length 2 (min_len)
 - Max length not set (max_len)
 - Numerals are removed (numerals, -N)
 - Symbols are removed (symbols, -S)

Use the `vectorize_corpus` script to create a new corpus with different settings.

The corpus is processed in the following ways when loaded:

 - Exclude tokens having a total word count less than `Min count`
 - Include at most `Top count` most frequent words.
 - Group and sum up documents by year.
 - Normalize token distribution over years to 1.0
 
 -->

In [ ]:
import word_trends_textacy_corpus_gui as textacy_corpus_gui
_ = textacy_corpus_gui.display_gui(corpus_folder, container=container)